In [12]:
import nest_asyncio
nest_asyncio.apply()

In [24]:
from crewai import Crew, Agent, Task, Process
from crewai.memory import ShortTermMemory, LongTermMemory, EntityMemory
from crewai.memory.storage.rag_storage import RAGStorage
from crewai.memory.storage.ltm_sqlite_storage import LTMSQLiteStorage

ltm_storage = LTMSQLiteStorage("ltm.db")
long_term_memory = LongTermMemory(storage=ltm_storage)
entity_memory = EntityMemory()
rag_storage = RAGStorage(path="short_term_memory.db", type="short_term")
short_term_memory = ShortTermMemory(storage=rag_storage)

assistant = Agent(
    name="Assistant",
    role="Personnel Assistant.",
    goal="""You are a personal assistant that can
                          help the user with their tasks.""",
    backstory="""You are a personal assistant that
                               can help the user with their tasks.""",
    verbose=False
)   
task = Task(
    name="Manage Daily Tasks",
    description="Handles this task:{user_task}.",
    expected_output="A clear and concise answer to the question.",
    agent=assistant
)

crew = Crew(
    name="Personal Assistant Crew", 
    agents=[assistant],
    tasks=[task],
    processes=Process.sequential,
    memory=True,
    short_term_memory=short_term_memory,
    long_term_memory=long_term_memory,
    entity_memory=entity_memory,
    verbose=False
)
user_input = """My favorite color is #46778F and
                my favorite Agent framework is CrewAI."""


result = crew.kickoff(inputs={"user_task": user_input})
print("Final Result:\n", result)
print("Long Term Memory Search:\n", crew._long_term_memory.search(task.description))
print("Entity Memory Search:\n", crew._entity_memory.search(user_input))
print("Short Term Memory Search:\n", crew._short_term_memory.search(user_input))

Final Result:
 Your favorite color is #46778F, which is a specific shade of blue represented by the hexadecimal color code. Your favorite Agent framework is CrewAI, indicating that you prefer to use the CrewAI framework for AI agent-related tasks or projects.
Long Term Memory Search:
 [{'metadata': {'suggestions': ['Request clear criteria for evaluation to ensure alignment with expectations.', 'Clarify the purpose of the task to tailor the response appropriately.', 'Encourage providing contextual or additional information about mentioned entities for better insights.'], 'quality': 8.0, 'agent': 'Personnel Assistant.', 'expected_output': 'A clear and concise answer to the question.'}, 'datetime': '1768709407.058889', 'score': 8.0}, {'metadata': {'suggestions': ["Ensure the output is concise and directly addresses the user's input without unnecessary preamble.", 'Include explanations only if explicitly requested or if it adds value to the response.', "Maintain a professional tone by avoi

In [8]:
crew._short_term_memory

ShortTermMemory(embedder_config=None, crew=None, storage=<crewai.memory.storage.rag_storage.RAGStorage object at 0x0000020EBACBD040>)

In [20]:
crew._short_term_memory.storage.search("what is my favorite color?")

[{'id': '4899e5c4ad3b65b364b476c8252abc3f787e5ee2b47d2cc786785397420b9243',
  'content': 'Thought: I now can give a great answer\nFinal Answer: Your favorite color is #46778F, which is a shade of blue with a hexadecimal color code. Additionally, your favorite Agent framework is CrewAI, indicating a preference for this specific AI agent framework in your tasks or projects.',
  'metadata': {'observation': 'Handles this task:My favorite color is #46778F and\n                my favorite Agent framework is CrewAI..'},
  'score': 0.69030891100923}]

In [15]:
result = crew.kickoff(inputs={"user_task": "what is my favorite color?"})

In [16]:
result.raw


'Your favorite color is #46778F.'

In [21]:
crew._long_term_memory.search(task.description)

[{'metadata': {'suggestions': ['Ensure the task prompt explicitly asks for a summary or recognition of provided preferences to guide the response.',
    'Provide clearer instructions on the expected format for the answer, such as avoiding meta-cognitive or internal thoughts in the final output.',
    'Request the response to explicitly identify and list entities for better extraction and clarity.'],
   'quality': 8.0,
   'agent': 'Personnel Assistant.',
   'expected_output': 'A clear and concise answer to the question.'},
  'datetime': '1768693504.5407698',
  'score': 8.0}]

In [18]:
user_input = "What is my favorite color?"
crew._entity_memory.search(user_input)

[{'id': '4f9d9e50498478db8de0496a4e858942b30dc52bbcb63455b8248eea47fe3f65',
  'content': '#46778F(Color): The favorite color specified in the task.',
  'metadata': {'relationships': ''},
  'score': 0.6915578391834833},
 {'id': 'a2abe45f0547dd70b5bd4d1c08d1520b00ea4a64f215abfb3b60b330ac6ae0cf',
  'content': '#46778F(Color): A hex color code representing a specific favorite color used for visual consistency in the schedule and task list.',
  'metadata': {'relationships': '- Visual theme'},
  'score': 0.6766781913302817},
 {'id': 'b7bd9364cb9b70890b0a361f25c4fdf468b04bd6de4f71992dfb06f6ef0096c2',
  'content': "#46778F(Color): The user's favorite color used for visual consistency in the schedule.",
  'metadata': {'relationships': ''},
  'score': 0.6751201333788747},
 {'id': '612ff1c695ba2c17dbfc78da5882133925ec456426a4fa9e09c9d3f0a588f387',
  'content': "#46778F(Color): User's favorite color used for visual consistency in the schedule.",
  'metadata': {'relationships': '- Used in schedule 

In [25]:
import sqlite3

conn = sqlite3.connect("ltm.db")
cursor = conn.cursor()
cursor.execute("SELECT * FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
for table in tables:
    print(table)
conn.close()

('table', 'long_term_memories', 'long_term_memories', 2, 'CREATE TABLE long_term_memories (\n                        id INTEGER PRIMARY KEY AUTOINCREMENT,\n                        task_description TEXT,\n                        metadata TEXT,\n                        datetime TEXT,\n                        score REAL\n                    )')
('table', 'sqlite_sequence', 'sqlite_sequence', 3, 'CREATE TABLE sqlite_sequence(name,seq)')


In [26]:
import sqlite3

conn = sqlite3.connect("ltm.db")
cursor = conn.cursor()
cursor.execute("SELECT * FROM long_term_memories;")
rows = cursor.fetchall()
for row in rows:
    print(row)
conn.close()

(1, 'Handles this task:My favorite color is #46778F and\n                my favorite Agent framework is CrewAI..', '{"suggestions": ["Ensure the task prompt explicitly asks for a summary or recognition of provided preferences to guide the response.", "Provide clearer instructions on the expected format for the answer, such as avoiding meta-cognitive or internal thoughts in the final output.", "Request the response to explicitly identify and list entities for better extraction and clarity."], "quality": 8.0, "agent": "Personnel Assistant.", "expected_output": "A clear and concise answer to the question."}', '1768693504.5407698', 8.0)
(2, 'Handles this task:My favorite color is #46778F and\n                my favorite Agent framework is CrewAI..', '{"suggestions": ["Ensure the response directly addresses the key points in the task description without unnecessary preamble.", "Provide brief explanations or context for technical terms or codes (e.g., hexadecimal color code) to enhance clari